In [1]:
%config Completer.use_jedi = False
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from glob import glob
import os

from sklearn.preprocessing import StandardScaler



In [2]:
def prepare_data_train(fname):
    """ read and prepare training data """
    # Read data
    data = pd.read_csv(fname)
    # events file
    events_fname = fname.replace('_data','_events')
    # read event file
    labels= pd.read_csv(events_fname)
    clean=data.drop(['id' ], axis=1)#remove id
    labels=labels.drop(['id' ], axis=1)#remove id
    return  clean,labels

In [3]:
def prepare_data_test(fname):
    """ read and prepare test data """
    # Read data
    data = pd.read_csv(fname)
    return data

scaler= StandardScaler()
def data_preprocess_train(X):
    X_prep=scaler.fit_transform(X)
    #do here your preprocessing
    return X_prep
def data_preprocess_test(X):
    X_prep=scaler.transform(X)
    #do here your preprocessing
    return X_prep


In [4]:
subsample =100

In [5]:
cols = ['HandStart','FirstDigitTouch',
        'BothStartLoadPhase','LiftOff',
        'Replace','BothReleased']

In [6]:
subjects = range(1,13)
ids_tot = []
pred_tot = []
nnpred_tot = []

In [7]:
for subject in subjects:
    y_raw= []
    raw = []
    ################ READ DATA ################################################
    fnames =  glob('../raw_date/train/subj%d_series*_data.csv' % (subject))
    print(fnames)
    for fname in fnames:
        data, labels=prepare_data_train(fname)
        raw.append(data)
        y_raw.append(labels)

    X = pd.concat(raw)
    y = pd.concat(y_raw)
    #transform in numpy array
    #transform train data in numpy array
    X_train =np.asarray(X.astype(float))
    y = np.asarray(y.astype(float))


    ################ Read test data #####################################
    #
    fnames =  glob('../raw_date/test/subj%d_series*_data.csv' % (subject))
    print(fnames)
    test = []
    idx=[]
    for fname in fnames:
        data=prepare_data_test(fname)
        test.append(data)
        idx.append(np.array(data['id']))
    X_test= pd.concat(test)
    ids=np.concatenate(idx)
    ids_tot.append(ids)
    X_test=X_test.drop(['id' ], axis=1)#remove id
    #transform test data in numpy array
    X_test =np.asarray(X_test.astype(float))


['../raw_date/train/subj1_series6_data.csv', '../raw_date/train/subj1_series7_data.csv', '../raw_date/train/subj1_series1_data.csv', '../raw_date/train/subj1_series5_data.csv', '../raw_date/train/subj1_series4_data.csv', '../raw_date/train/subj1_series2_data.csv', '../raw_date/train/subj1_series3_data.csv', '../raw_date/train/subj1_series8_data.csv']
['../raw_date/test/subj1_series10_data.csv', '../raw_date/test/subj1_series9_data.csv']
['../raw_date/train/subj2_series8_data.csv', '../raw_date/train/subj2_series3_data.csv', '../raw_date/train/subj2_series2_data.csv', '../raw_date/train/subj2_series4_data.csv', '../raw_date/train/subj2_series5_data.csv', '../raw_date/train/subj2_series1_data.csv', '../raw_date/train/subj2_series7_data.csv', '../raw_date/train/subj2_series6_data.csv']
['../raw_date/test/subj2_series9_data.csv', '../raw_date/test/subj2_series10_data.csv']
['../raw_date/train/subj3_series5_data.csv', '../raw_date/train/subj3_series4_data.csv', '../raw_date/train/subj3_seri

In [10]:
len(X_train)

1607923

In [16]:
X_train.shape

(1607923, 32)

In [12]:
y.shape

(1607923, 6)

In [13]:
X_test.shape

(287869, 32)

### visuals MNE

In [1]:
data = np.loadtxt('/content/drive/MyDrive/train/subj1_series1_data.csv', delimiter=',', usecols =range(1,33), skiprows=1)
labels = np.loadtxt('/content/drive/MyDrive/train/subj1_series1_events.csv', delimiter=',', usecols=range(1,7), skiprows=1)

NameError: name 'np' is not defined

In [ ]:
ch_names= ['Fp1',	'Fp2',	'F7',	'F3',	'Fz',	'F4',	'F8',	'FC5',	'FC1',	'FC2',	'FC6',	'T7',	'C3',	'Cz','C4',	'T8',	'TP9',	'CP5',	'CP1',	'CP2',	'CP6',	'TP10',	'P7',	'P3',	'Pz',	'P4',	'P8',	'PO9',	'O1',	'Oz',	'O2',	'PO10',"Handstart (HS)","Grasping (GS)","Lift (LT)","Hold (HD)","Replace (RP)","Release (RL)"]
sfreq =500
info = mne.create_info(ch_names, sfreq)

In [ ]:
lab_names=["Handstart (HS)","Grasping (GS)","Lift (LT)","Hold (HD)","Replace (RP)","Release (RL)"]
labels_T=np.transpose(labels)

In [ ]:
data_T=np.transpose(data)
data_df= pd.DataFrame(data_T)
labels_df= pd.DataFrame(labels_T)

In [ ]:
table=pd.concat([data_df,labels_df],axis=0,)

In [ ]:
raw = mne.io.RawArray(table, info)

In [ ]:
raw.plot()

In [ ]:
events = mne.find_events(raw, stim_channel='C3',initial_event=True, min_duration=0.002, uint_cast=True,shortest_event=1)

In [ ]:
event_dict = {"Handstart (HS)":1,"Grasping (GS)":2,"Lift (LT)":3,"Hold (HD)":4,"Replace (RP)":5,"Release (RL)":6}
epochs = mne.Epochs(raw, events, tmin=-0.3, tmax=0.7, event_id=event_dict,
                    preload=True)
evoked = epochs['Handstart (HS)'].average()